# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
!pip install gmaps

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
# Load the csv exported in Part I to a DataFrame

weather_csv_file = "./output_data/city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port alfred,-33.5906,26.8910,65.77,30.0,99.0,8.90,ZA,1.627776e+09
1,hilo,19.7297,-155.0900,80.11,78.0,60.0,8.01,US,1.627776e+09
2,mahenge,-8.6833,36.7167,61.90,63.0,93.0,3.83,TZ,1.627776e+09
3,saldanha,-33.0117,17.9442,55.45,68.0,36.0,6.26,ZA,1.627776e+09
4,butaritari,3.0707,172.7902,82.08,74.0,38.0,8.93,KI,1.627776e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
weather_df["Humidity"].dtypes

dtype('float64')

In [172]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

humidity = weather_df["Humidity"]
locations = weather_df[["Lat", "lng"]]

fig = gmaps.figure(center = [0,0], zoom_level=4)
heatmap_layer = gmaps.heatmap_layer(locations, \
                weights = humidity, max_intensity=300, point_radius=10)

fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
# Choosing favorable temperature 70F to 80F 
good_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)]

# Choosing wind speed less than 10
good_weather_df = good_weather_df.loc[good_weather_df["Wind Speed"] < 10]

# Choosing no cloudiness (clear sky)
good_weather_df = good_weather_df.loc[good_weather_df["Cloudiness"] == 0]

# Dropping any NAs
good_weather_df = good_weather_df.dropna()

# New data for searching cities as desired 
good_weather_df


,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,ballina,-28.8667,153.5667,71.78,59.0,0.0,7.85,AU,1.627776e+09
55,manono,-7.3000,27.4167,72.48,33.0,0.0,6.98,CD,1.627776e+09
101,kyrylivka,46.3759,35.3675,77.63,57.0,0.0,5.30,UA,1.627776e+09
164,abu samrah,35.3029,37.1841,77.79,50.0,0.0,6.73,SY,1.627776e+09
170,bucak,37.4592,30.5950,77.04,33.0,0.0,7.07,TR,1.627776e+09
186,saskylakh,71.9167,114.0833,75.92,34.0,0.0,9.55,RU,1.627776e+09
300,moranbah,-22.0016,148.0466,73.35,59.0,0.0,4.99,AU,1.627776e+09
361,severnyy,50.6770,36.5532,71.64,64.0,0.0,0.00,RU,1.627776e+09
375,bagotville,-28.9833,153.4167,73.42,58.0,0.0,5.79,AU,1.627776e+09
451,uyutne,45.1994,33.3014,72.82,68.0,0.0,5.46,UA,1.627776e+09


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [199]:
hotel_df = good_weather_df.loc[:,["Lat","lng","City","Country"]]

#setting up lists for hotel information
hotel_name = []
hotel_address = []
hotel_rating = []
hotel_lat = []
hotel_lng = []

print("Beginning Hotel data Retrieval")
print("♦" * 45)


for index, row in hotel_df.iterrows():
    
    current_city = row["City"]
    current_country = row["Country"]
    
    hotel_location = f"{row['Lat']},{row['lng']}"
    hotel_radius = 5000 #this is in meters
#     hotel_type = "hotel"
    hotel_keyword = "hotel"

    #set parameters
    params = {
        "location": hotel_location,
        "radius": hotel_radius,
#         "type": hotel_type,
        "keyword": hotel_keyword,
        "key": g_key }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url,params=params).json()
    results = response['results']
     
    try:
        hotel_name.append(results[0]['name'])
        hotel_address.append(results[0]['vicinity'])
        hotel_rating.append(results[0]['rating'])
        hotel_lat.append(results[0]['geometry']['location']['lat'])
        hotel_lng.append(results[0]['geometry']['location']['lng'])
        print(f"Processeing Record for a {hotel_keyword} in {current_city}, {current_country}")

    except:
        hotel_name.append(np.nan)
        hotel_address.append(np.nan)
        hotel_rating.append(np.nan)
        hotel_lat.append(np.nan)
        hotel_lng.append(np.nan)
        print(f"No results/{hotel_keyword} found in {current_city}, {current_country}. Skipping...")

print("♦" * 45)
print("Hotel data Retrieval is Complete")
print("♦" * 45)


Beginning Hotel data Retrieval
♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦
Processeing Record for a hotel in ballina, AU
Processeing Record for a hotel in manono, CD
Processeing Record for a hotel in kyrylivka, UA
No results/hotel found in abu samrah, SY. Skipping...
Processeing Record for a hotel in bucak, TR
Processeing Record for a hotel in saskylakh, RU
Processeing Record for a hotel in moranbah, AU
Processeing Record for a hotel in severnyy, RU
No results/hotel found in bagotville, AU. Skipping...
Processeing Record for a hotel in uyutne, UA
No results/hotel found in korisos, GR. Skipping...
Processeing Record for a hotel in lichuan, CN
♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦
Hotel data Retrieval is Complete
♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦♦


In [200]:
# add new information into the cities_df
hotel_df["Hotel Name"] = hotel_name
hotel_df["Hotel Address"] = hotel_address
hotel_df["Hotel Rating"] = hotel_rating
hotel_df["Hotel Latitude"] = hotel_lat
hotel_df["Hotel Longitude"] = hotel_lng

hotel_df

,Lat,lng,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
41,-28.8667,153.5667,ballina,AU,Ramada Hotel & Suites by Wyndham Ballina Byron,"2 Martin St, Ballina",4.3,-28.872736,153.562446
55,-7.3000,27.4167,manono,CD,Hotel hewa bora,"P94W+V84, Manono",0.0,-7.292855,27.395825
101,46.3759,35.3675,kyrylivka,UA,EGOIST - Отельный комплекс на берегу Азовского...,"Fedotova kosa St, 165, Kyrylivka",4.2,46.329355,35.332473
164,35.3029,37.1841,abu samrah,SY,NaN,NaN,NaN,NaN,NaN
170,37.4592,30.5950,bucak,TR,Ada Hotel,"Oğuzhan, Bucak Burdur TR, Ramazan Selen Blv. N...",4.6,37.458384,30.593975
186,71.9167,114.0833,saskylakh,RU,Sberbank,"ул. Мира, 6, Saskylakh",3.5,71.962965,114.092010
300,-22.0016,148.0466,moranbah,AU,Oaks Moranbah Suites,"11 Bacon St, Moranbah",4.1,-22.002072,148.042501
361,50.6770,36.5532,severnyy,RU,Hotel VOYAGE,"Bogdana Khmel'nitskogo Prospekt, 155, Belgorod",4.6,50.639663,36.568763
375,-28.9833,153.4167,bagotville,AU,NaN,NaN,NaN,NaN,NaN
451,45.1994,33.3014,uyutne,UA,Tsar' Yevpator,"64A, Pushkina St, 64А, Yevpatoriya",4.2,45.186850,33.352267


In [201]:
#remove NaN rows
#did not drop indexes for locations without hotel found - can see how many hotels are found for top locations
hotel_found_df = hotel_df.dropna(axis = 0)
hotel_found_df

,Lat,lng,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
41,-28.8667,153.5667,ballina,AU,Ramada Hotel & Suites by Wyndham Ballina Byron,"2 Martin St, Ballina",4.3,-28.872736,153.562446
55,-7.3000,27.4167,manono,CD,Hotel hewa bora,"P94W+V84, Manono",0.0,-7.292855,27.395825
101,46.3759,35.3675,kyrylivka,UA,EGOIST - Отельный комплекс на берегу Азовского...,"Fedotova kosa St, 165, Kyrylivka",4.2,46.329355,35.332473
170,37.4592,30.5950,bucak,TR,Ada Hotel,"Oğuzhan, Bucak Burdur TR, Ramazan Selen Blv. N...",4.6,37.458384,30.593975
186,71.9167,114.0833,saskylakh,RU,Sberbank,"ул. Мира, 6, Saskylakh",3.5,71.962965,114.092010
300,-22.0016,148.0466,moranbah,AU,Oaks Moranbah Suites,"11 Bacon St, Moranbah",4.1,-22.002072,148.042501
361,50.6770,36.5532,severnyy,RU,Hotel VOYAGE,"Bogdana Khmel'nitskogo Prospekt, 155, Belgorod",4.6,50.639663,36.568763
451,45.1994,33.3014,uyutne,UA,Tsar' Yevpator,"64A, Pushkina St, 64А, Yevpatoriya",4.2,45.186850,33.352267
566,30.3000,108.8500,lichuan,CN,Yicheng Hotel,"Zhutian Village, Lichuan",0.0,30.286620,108.815950


In [202]:
print(f"A total of {len(hotel_df)} locations satisfy the required conditions of interst for an ideal vacation place \
and {len(hotel_found_df)} hotels found.")



A total of 12 locations satisfy the required conditions of interst for an ideal vacation place and 9 hotels found.


In [203]:
# NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>Ratings</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


In [204]:
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_found_df.iterrows()]
locations = hotel_found_df[["Hotel Latitude", "Hotel Longitude"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))